In [666]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

%matplotlib inline

# DataFrame

## Create DataFrame

pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

data : numpy ndarray (structured or homogeneous), dict, or DataFrame. Dict can contain Series, arrays, constants, or list-like objects

index : Index or array-like Index to use for resulting frame. Will default to RangeIndex if no indexing information part of input data and no index provided

dtype : dtype, default None. Data type to force. Only a single dtype is allowed. If None, infer

columns : Index or array-like Column labels to use for resulting frame. Will default to RangeIndex (0, 1, 2, ..., n) if no column labels are provided

### From Dictionary

In [667]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d, index=['a', 'b'], dtype=np.int8)
df

,col1,col2
a,1,3
b,2,4


In [668]:
df.dtypes # If None, inferred dtype is int64.

col1    int8
col2    int8
dtype: object

### From numpy ndarray

In [669]:
df2 = pd.DataFrame(np.random.randint(low=0, high=10, size=(10, 5)), columns=['a', 'b', 'c', 'd', 'e'])
df2

,a,b,c,d,e
0,2,4,7,0,5
1,6,3,5,2,4
2,2,8,5,7,9
3,6,8,2,5,7
4,7,4,9,6,2
5,8,9,7,4,7
6,0,7,6,0,1
7,2,6,5,6,5
8,6,1,2,9,2
9,5,2,9,2,1


### From list

In [670]:
data = [[12, 2], [0, 4], [10, 20], [1, 4], [7, 1], [16, 36]] # data list
tuples = [('cobra', 'mark i'), ('cobra', 'mark ii'),('sidewinder', 'mark i'), 
          ('sidewinder', 'mark ii'),('viper', 'mark ii'), ('viper', 'mark iii')]
index = pd.MultiIndex.from_tuples(tuples)
index

MultiIndex(levels=[['cobra', 'sidewinder', 'viper'], ['mark i', 'mark ii', 'mark iii']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 1, 2]])

In [671]:
df = pd.DataFrame(data, columns=['max_speed', 'sheild'], index=index)
df

max_speed  sheild
cobra      mark i           12       2
           mark ii           0       4
sidewinder mark i           10      20
           mark ii           1       4
viper      mark ii           7       1
           mark iii         16      36

## DataFrame Methods

In [672]:
d = list(range(10, 13))
df = pd.DataFrame(d)
df

,0
0,10
1,11
2,12


### column, index

In [673]:
df.columns

RangeIndex(start=0, stop=1, step=1)

In [674]:
df.columns = ['col1'] # rename column(s)
df

,col1
0,10
1,11
2,12


In [675]:
df.index

RangeIndex(start=0, stop=3, step=1)

In [676]:
df.index = ['a', 'b', 'c']
df

,col1
a,10
b,11
c,12


In [677]:
df = df.set_index(['col1'])
df.index

Int64Index([10, 11, 12], dtype='int64', name='col1')

#### Add new columns

In [678]:
df['new_col'] = 13 # add new column. Can't use df.new_col
df

,new_col
col1,
10,13
11,13
12,13


### Select 

#### Select vertical pieces

In [679]:
df.new_col # pandas.core.series.Series # Here we can use df.new_col

col1
10    13
11    13
12    13
Name: new_col, dtype: int64

In [680]:
df[['new_col', 'col1']]

KeyError: "['col1'] not in index"

#### Select horizontal pieces

In [ ]:
df.loc['a'] # pandas.core.series.Series

In [ ]:
df.loc['a':'b'] # df.loc[inclusive:inclusive]

#### Select both vertically and horizontally

In [ ]:
df.loc['a', 'new_col'] # numpy.int64

In [ ]:
df.loc['a', 'col1':'new_col'] # pandas.core.series.Series

In [ ]:
df.loc['a':'b', 'col1':'new_col'] # pandas.core.frame.DataFrame ## 'a':'b' can be replaced by df.index[0:2]

In [ ]:
df.loc[['a','c'], ['col1', 'new_col']] # pandas.core.frame.DataFrame

#### Select by boolean array

In [ ]:
df.loc[[True, True, False]] # select first two rows

In [ ]:
df.loc[df['col1']>10] # select rows whose col1 > 10

In [ ]:
df.loc[lambda df: df['col1']>10, ['new_col']] # select rows whose rol1>10 and show their new_col column

### Stack, Unstack

In [ ]:
stack = df.stack() # Bring the columns and place them in the index
stack

In [ ]:
stack.index # # The index now includes the column names

In [ ]:
unstack = df.unstack()
unstack

In [ ]:
unstack.index

In [ ]:
df = stack.unstack()
df

In [ ]:
df.T # exchange columns and indices

### groupby

In [ ]:
d = {'num1':[1,1,2,2,3,3],
     'num2':[1,2,3,1,2,3],
     'letter':['a','a','a','b','b','b']}

df = pd.DataFrame(d)
df

In [ ]:
letter = df.groupby(['letter']).sum()
letter

In [ ]:
letter_num = df.groupby(['letter', 'num1']).sum()
letter_num

In [ ]:
letter_num.index

In [ ]:
letter_num = df.groupby(['letter', 'num1'], as_index=False).sum()
letter_num

In [ ]:
letter_num.index

# Application

## Calc for outliers

In [ ]:
States = ['NY', 'NY', 'NY', 'NY', 'FL', 'FL', 'GA', 'GA', 'FL', 'FL'] 
data = [1.0, 2, 3, 4, 5, 6, 7, 8, 9, 10]
idx = pd.date_range('1/1/2012', periods=10, freq='MS')
df1 = pd.DataFrame(data, index=idx, columns=['Revenue'])
df1['State'] = States

data2 = [10.0, 10.0, 9, 9, 8, 8, 7, 7, 6, 6]
idx2 = pd.date_range('1/1/2013', periods=10, freq='MS')
df2 = pd.DataFrame(data2, index=idx2, columns=['Revenue'])
df2['State'] = States

df = pd.concat([df1, df2])
df

### Method 1: groupby.transform

In [ ]:
newdf = df.copy()
StateMonth = newdf.groupby(['State', lambda x:x.month])
StateMonth.sum()

In [ ]:
newdf['x-Mean'] = StateMonth.transform(lambda x: abs(x-x.mean()))
newdf['1.96*std'] = StateMonth.transform(lambda x: 1.96*x.std())
newdf['Outlier'] = StateMonth.transform(lambda x: abs(x-x.mean()) > 1.96*x.std())
newdf

### Method 2: groupby.apply

In [ ]:
newdf = df.copy()

StateMonth = newdf.groupby(['State', lambda x:x.month])

def s(group):
    group['x-Mean'] = abs(group['Revenue'] - group['Revenue'].mean())
    group['1.96*std'] = 1.96 * group['Revenue'].std()
    group['Outlier'] = group['x-Mean'] > group['1.96*std']
    return group

newdf2 = StateMonth.apply(s)
newdf2

### Compare groupby.transform and groupby.apply

#### groupby.transform

In [ ]:
df_ = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar','foo', 'bar'],
                   'B' : ['one', 'one', 'two', 'three','two', 'two'],
                   'C' : [1, 5, 5, 2, 5, 5],
                   'D' : [2.0, 5., 8., 1., 2., 9.]})
df_

In [ ]:
grouped = df_.groupby('A')
grouped.transform(lambda x: (x - x.mean()) / x.std()) # @1

@1: Call function producing a like-indexed Series on **each group** and return a Series having the same indexes as the original object filled with the transformed values

####  groupby.apply 

In [ ]:
ser = pd.Series([0, 1, 2], index='a a b'.split())
ser

In [ ]:
ser.index

In [ ]:
g = ser.groupby(ser.index)
g.apply(lambda x:  x*2 if x.name == 'b' else x/2) # @1

@1: Apply function ``func``  group-wise and combine the results together.
The function passed to ``apply`` must take a series as its first
argument and return a dataframe, a series or a scalar. ``apply`` will
then take care of combining the results back together into a single
dataframe or series. ``apply`` is therefore a highly flexible
grouping method.

In my opinion, `transform` can get integral property of a group of numbers (such as mean, std, and anything), while `apply` can only do calc on individual number in a group.

### Method 3: Consider all data

In [ ]:
newdf = df.copy()

newdf['x-Mean'] = abs(newdf['Revenue'] - newdf['Revenue'].mean())
newdf['1.96*std'] = 1.96*newdf['Revenue'].std()
newdf['Outlier'] = newdf['x-Mean'] > newdf['1.96*std']
newdf

### If non-Gaussian distribution: 

In [ ]:
# make a copy of original df
newdf = df.copy()

State = newdf.groupby('State')

newdf['Lower'] = State['Revenue'].transform(lambda x: x.quantile(q=.25) 
                                            - (1.5*(x.quantile(q=.75)-x.quantile(q=.25))) )
newdf['Upper'] = State['Revenue'].transform(lambda x: x.quantile(q=.75) 
                                            + (1.5*(x.quantile(q=.75)-x.quantile(q=.25))) )
newdf['Outlier'] = (newdf['Revenue'] < newdf['Lower']) | (newdf['Revenue'] > newdf['Upper']) 
newdf

# Export Data

In [ ]:
# df.to_csv('data.csv') # columns=None, header=True, index=True, index_label=None
# df.to_csv('data.txt') 
# df.to_excel('data.xls') # sheet_name='Sheet1', columns=None, header=True, index=True, 
                          # index_label=None, startrow=0, startcol=0
# df.to_json('data.json')

# Load Data

In [ ]:
# pd.read_csv('data.csv') # header='infer', names=None, index_col=None
# pd.read_csv('data.txt') 
# pd.read_excel('data.xls') # sheet_name=0, header=0, names=None, index_col=None
# pd.read_json('data.json') 